In [35]:
import pygame
import numpy as np

# --- Pygame 초기화 ---
pygame.init() # Pygame 라이브러리 초기화
WIDTH, HEIGHT = 800, 600  # 화면 너비와 높이 설정
screen = pygame.display.set_mode((WIDTH, HEIGHT)) # 지정된 크기의 화면 생성
pygame.display.set_caption("2D 칼만 필터 시뮬레이션 (측정 단계 이동)") # 창 제목 설정
clock = pygame.time.Clock() # 프레임 속도 제어를 위한 Clock 객체 생성

# --- 시뮬레이션 파라미터 ---
dt = 1.0  # 시간 간격 (델타 t)
accel_noise_std = 0.2  # 가속도 잡음(프로세스 잡음)의 표준 편차
measurement_noise_std = 5.0  # 측정 잡음(센서 잡음)의 표준 편차
num_steps = 300 # 총 시뮬레이션 스텝 수
accel_range = 0.05 # 가속도 최대 크기

# --- 시스템 상태 정의 ---
# 상태 벡터: [x_위치, y_위치, x_속도, y_속도]
A = np.array([[1, 0, dt, 0],
              [0, 1, 0, dt],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
B = np.array([[0.5 * dt**2, 0],
              [0, 0.5 * dt**2],
              [dt, 0],
              [0, dt]])
H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0]])
Q = np.eye(4) * accel_noise_std**2
R = np.eye(2) * measurement_noise_std**2

# --- 초기 실제 상태 ---
initial_true_state = np.array([WIDTH / 3.0, HEIGHT / 3.0, 0.5, 0.3]) # [x, y, vx, vy]

# --- 칼만 필터 초기 추정값 ---
x_est_initial = np.array([WIDTH / 3.0 - 50, HEIGHT / 3.0 - 50, 0.0, 0.0]) # 초기 추측값

# --- 초기 추정 공분산 ---
P_initial = np.eye(4) * 10.0 # 초기 불확실성

# ==============================================================
# 단계 1: 실제 경로 및 제어 입력 시뮬레이션
# ==============================================================
print("단계 1: 실제 경로 및 제어 입력 시뮬레이션 중...")

true_states = []      # 각 시간 스텝에서의 실제 상태를 저장할 리스트
control_inputs = []   # 사용된 제어 입력(임의 가속도)을 저장할 리스트

current_true_state = initial_true_state.copy()

for k in range(num_steps):
    # --- 임의의 제어 입력 (가속도) 생성 ---
    ax = np.random.uniform(-accel_range, accel_range)
    ay = np.random.uniform(-accel_range, accel_range)
    u = np.array([ax, ay])

    # --- 프로세스 잡음 생성 ---
    process_noise = np.random.multivariate_normal([0, 0, 0, 0], Q)

    # --- 실제 상태 업데이트 ---
    current_true_state = A @ current_true_state + B @ u + process_noise

    # --- 결과 저장 (실제 상태와 제어 입력만) ---
    true_states.append(current_true_state.copy())
    control_inputs.append(u.copy())

print(f"단계 1 완료: {num_steps} 스텝의 실제 경로 및 제어 입력 생성됨.")

# ==============================================================
# 단계 2: 칼만 필터 적용 (측정값 실시간 생성)
# ==============================================================
print("단계 2: 칼만 필터 적용 및 측정값 실시간 생성 중...")

estimated_states = []       # 각 스텝에서의 칼만 필터 상태 추정값을 저장할 리스트
measurements_for_viz = []   # 시각화를 위해 이 단계에서 생성된 측정값을 저장할 리스트

# 칼만 필터 상태 및 공분산 초기화
x_est = x_est_initial.copy()
P = P_initial.copy()

for k in range(num_steps):
    # 현재 시간 스텝의 실제 상태와 제어 입력을 가져옵니다.
    true_state_k = true_states[k] # <-- 현재 스텝의 실제 상태 필요
    u_k = control_inputs[k]

    # --- 예측 단계 (Prediction Step) ---
    x_pred = A @ x_est + B @ u_k
    P_pred = A @ P @ A.T + Q

    # --- 측정값 실시간 생성 ---
    # 현재 실제 상태(true_state_k)를 기반으로 측정 잡음을 추가하여 측정값을 생성합니다.
    measurement_noise = np.random.multivariate_normal([0, 0], R)
    measurement_k = H @ true_state_k + measurement_noise # <-- 여기서 측정값 생성
    measurements_for_viz.append(measurement_k.copy()) # 시각화를 위해 저장

    # --- 갱신 단계 (Update Step) ---
    # 방금 생성한 measurement_k를 사용합니다.
    y = measurement_k - H @ x_pred
    S = H @ P_pred @ H.T + R
    K = P_pred @ H.T @ np.linalg.inv(S)
    x_est = x_pred + K @ y
    P = (np.eye(4) - K @ H) @ P_pred

    # --- 결과 저장 ---
    estimated_states.append(x_est.copy()) # 갱신된 추정값 저장

print("단계 2 완료: 칼만 필터 적용 및 측정값 생성 완료.")

# ==============================================================
# 단계 3: 시각화
# ==============================================================
print("단계 3: 시각화 시작...")

running = True
frame_index = 0

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255, 255, 255))

    if frame_index < num_steps:
        # 현재 프레임 데이터 가져오기
        true_state_k = true_states[frame_index]
        measurement_k = measurements_for_viz[frame_index] # <-- 단계 2에서 저장한 측정값 사용
        estimated_state_k = estimated_states[frame_index]

        # 실제 위치 (파란색)
        true_x = int(np.clip(true_state_k[0], 0, WIDTH))
        true_y = int(np.clip(true_state_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, (0, 0, 255), (true_x, true_y), 8)

        # 측정 위치 (회색)
        meas_x = int(np.clip(measurement_k[0], 0, WIDTH))
        meas_y = int(np.clip(measurement_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, (180, 180, 180), (meas_x, meas_y), 6)

        # 추정 위치 (빨간색)
        est_x = int(np.clip(estimated_state_k[0], 0, WIDTH))
        est_y = int(np.clip(estimated_state_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, (255, 0, 0), (est_x, est_y), 8)

        # 궤적 선 그리기
        if frame_index > 0:
             prev_true_x = int(np.clip(true_states[frame_index-1][0], 0, WIDTH))
             prev_true_y = int(np.clip(true_states[frame_index-1][1], 0, HEIGHT))
             pygame.draw.line(screen, (0, 0, 200), (prev_true_x, prev_true_y), (true_x, true_y), 1)
             prev_est_x = int(np.clip(estimated_states[frame_index-1][0], 0, WIDTH))
             prev_est_y = int(np.clip(estimated_states[frame_index-1][1], 0, HEIGHT))
             pygame.draw.line(screen, (200, 0, 0), (prev_est_x, prev_est_y), (est_x, est_y), 1)

        # 스텝 정보 표시
        font = pygame.font.Font(None, 30)
        text = font.render(f"스텝: {frame_index+1}/{num_steps}", True, (0, 0, 0))
        screen.blit(text, (10, 10))

        frame_index += 1
    else:
        # 완료 메시지
        font = pygame.font.Font(None, 50)
        text = font.render("시뮬레이션 완료", True, (0, 100, 0))
        text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text, text_rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()
print("시각화 종료 및 Pygame 종료됨.")

단계 1: 실제 경로 및 제어 입력 시뮬레이션 중...
단계 1 완료: 300 스텝의 실제 경로 및 제어 입력 생성됨.
단계 2: 칼만 필터 적용 및 측정값 실시간 생성 중...
단계 2 완료: 칼만 필터 적용 및 측정값 생성 완료.
단계 3: 시각화 시작...
시각화 종료 및 Pygame 종료됨.


In [25]:
import pygame
import numpy as np
import math

# --- Pygame 초기화 ---
pygame.init()
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D 칼만 필터 (구심 가속도 사용 원 궤도)") # 창 제목 변경
clock = pygame.time.Clock()

# --- 시뮬레이션 파라미터 ---
dt = 1.0
accel_noise_std = 0.1
measurement_noise_std = 5.0
num_steps = 300

# --- 원 궤도 파라미터 ---
center_x = WIDTH / 2.0
center_y = HEIGHT / 2.0
radius = 200.0
angular_velocity = 0.05 # 라디안/스텝

# --- 시스템 상태 정의 및 행렬 ---
A = np.array([[1, 0, dt, 0],
              [0, 1, 0, dt],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
B = np.array([[0.5 * dt**2, 0],
              [0, 0.5 * dt**2],
              [dt, 0],
              [0, dt]])
H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0]])
Q = np.eye(4) * accel_noise_std**2
R = np.eye(2) * measurement_noise_std**2

# --- 초기 실제 상태 (원 궤도 시작점으로 설정) ---
initial_angle = 0.0
initial_true_x = center_x + radius * math.cos(initial_angle)
initial_true_y = center_y + radius * math.sin(initial_angle)
initial_true_vx = -radius * angular_velocity * math.sin(initial_angle)
initial_true_vy = radius * angular_velocity * math.cos(initial_angle)
initial_true_state = np.array([initial_true_x, initial_true_y, initial_true_vx, initial_true_vy])

# --- 칼만 필터 초기 추정값 ---
x_est_initial = np.array([initial_true_x + 10, initial_true_y - 10, 0.0, 0.0])

# --- 초기 추정 공분산 ---
P_initial = np.eye(4) * 500.0

# ==========================================================================
# 단계 1: 구심 가속도를 이용한 실제 원 궤도 경로 시뮬레이션
# ==========================================================================
print("단계 1: 구심 가속도를 이용한 실제 원 궤도 경로 시뮬레이션 중...")

true_states = []      # 각 시간 스텝에서의 실제 상태 저장 리스트
control_inputs = []   # <-- 각 스텝에서 사용된 구심 가속도(제어 입력) 저장

current_true_state = initial_true_state.copy() # 현재 상태 초기화

for k in range(num_steps):
    # --- 현재 위치 가져오기 (이번 스텝 계산을 위해) ---
    x_k = current_true_state[0]
    y_k = current_true_state[1]

    # --- 구심 가속도 계산 ---
    # a = -omega^2 * (pos - center)
    ax_c = -angular_velocity**2 * (x_k - center_x)
    ay_c = -angular_velocity**2 * (y_k - center_y)
    u = np.array([ax_c, ay_c]) # 구심 가속도를 제어 입력 u로 사용

    # --- 프로세스 잡음 생성 ---
    process_noise = np.random.multivariate_normal([0, 0, 0, 0], Q)

    # --- 실제 상태 업데이트 (A, B 행렬 및 계산된 u 사용) ---
    # 다음 상태 = A @ 현재 상태 + B @ u + 프로세스 잡음
    current_true_state = A @ current_true_state + B @ u + process_noise

    # --- 결과 저장 ---
    true_states.append(current_true_state.copy())
    control_inputs.append(u.copy()) # 계산된 제어 입력(가속도) 저장

print(f"단계 1 완료: {num_steps} 스텝의 원 궤도 경로 및 제어 입력 생성됨.")

# ==========================================================================
# 단계 2: 칼만 필터 적용 (계산된 제어 입력 u 사용)
# ==========================================================================
print("단계 2: 칼만 필터 적용 (원 운동 제어 입력 사용)...")

estimated_states = []
measurements_for_viz = []

x_est = x_est_initial.copy()
P = P_initial.copy()

for k in range(num_steps):
    # 현재 시간 스텝의 실제 상태와 **제어 입력** 가져오기
    true_state_k = true_states[k]
    u_k = control_inputs[k] # <-- 단계 1에서 저장한 제어 입력 사용

    # --- 예측 단계 (Prediction Step) ---
    # 제어 입력 u_k를 사용하여 예측
    x_pred = A @ x_est + B @ u_k # <-- B @ u_k 포함하여 예측

    P_pred = A @ P @ A.T + Q

    # --- 측정값 실시간 생성 ---
    measurement_noise = np.random.multivariate_normal([0, 0], R)
    measurement_k = H @ true_state_k + measurement_noise
    measurements_for_viz.append(measurement_k.copy())

    # --- 갱신 단계 (Update Step) ---
    y = measurement_k - H @ x_pred
    S = H @ P_pred @ H.T + R
    K = P_pred @ H.T @ np.linalg.inv(S)
    x_est = x_pred + K @ y
    P = (np.eye(4) - K @ H) @ P_pred

    # --- 결과 저장 ---
    estimated_states.append(x_est.copy())

print("단계 2 완료: 칼만 필터 적용 완료.")

# ==============================================================
# 단계 3: 시각화 (변경 없음)
# ==============================================================
print("단계 3: 시각화 시작...")

running = True
frame_index = 0

# --- (시각화 루프 코드는 이전과 동일) ---
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255, 255, 255))

    if frame_index < num_steps:
        # 현재 프레임 데이터 가져오기
        true_state_k = true_states[frame_index]
        measurement_k = measurements_for_viz[frame_index]
        estimated_state_k = estimated_states[frame_index]

        # 실제 위치 (파란색)
        true_x = int(np.clip(true_state_k[0], 0, WIDTH))
        true_y = int(np.clip(true_state_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, (0, 0, 255), (true_x, true_y), 8)

        # 측정 위치 (회색)
        meas_x = int(np.clip(measurement_k[0], 0, WIDTH))
        meas_y = int(np.clip(measurement_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, (180, 180, 180), (meas_x, meas_y), 6)

        # 추정 위치 (빨간색)
        est_x = int(np.clip(estimated_state_k[0], 0, WIDTH))
        est_y = int(np.clip(estimated_state_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, (255, 0, 0), (est_x, est_y), 8)

        # 궤적 선 그리기
        if frame_index > 0:
             prev_true_x = int(np.clip(true_states[frame_index-1][0], 0, WIDTH))
             prev_true_y = int(np.clip(true_states[frame_index-1][1], 0, HEIGHT))
             pygame.draw.line(screen, (0, 0, 200), (prev_true_x, prev_true_y), (true_x, true_y), 1)
             prev_est_x = int(np.clip(estimated_states[frame_index-1][0], 0, WIDTH))
             prev_est_y = int(np.clip(estimated_states[frame_index-1][1], 0, HEIGHT))
             pygame.draw.line(screen, (200, 0, 0), (prev_est_x, prev_est_y), (est_x, est_y), 1)

        # 스텝 정보 표시
        font = pygame.font.Font(None, 30)
        text = font.render(f"스텝: {frame_index+1}/{num_steps}", True, (0, 0, 0))
        screen.blit(text, (10, 10))

        frame_index += 1
    else:
        # 완료 메시지
        font = pygame.font.Font(None, 50)
        text = font.render("시뮬레이션 완료", True, (0, 100, 0))
        text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text, text_rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()
print("시각화 종료 및 Pygame 종료됨.")

단계 1: 구심 가속도를 이용한 실제 원 궤도 경로 시뮬레이션 중...
단계 1 완료: 300 스텝의 원 궤도 경로 및 제어 입력 생성됨.
단계 2: 칼만 필터 적용 (원 운동 제어 입력 사용)...
단계 2 완료: 칼만 필터 적용 완료.
단계 3: 시각화 시작...
시각화 종료 및 Pygame 종료됨.


In [1]:
import pygame
import numpy as np

# --- Pygame 초기화 ---
pygame.init() # Pygame 라이브러리 초기화
WIDTH, HEIGHT = 800, 600  # 화면 너비와 높이 설정
screen = pygame.display.set_mode((WIDTH, HEIGHT)) # 지정된 크기의 화면 생성
pygame.display.set_caption("2D 칼만 필터 시뮬레이션 (이전 추정 기반 측정)") # 창 제목 수정
clock = pygame.time.Clock() # 프레임 속도 제어를 위한 Clock 객체 생성

# --- 시뮬레이션 파라미터 ---
dt = 1.0  # 시간 간격 (델타 t)
accel_noise_std = 0.2  # 가속도 잡음(프로세스 잡음)의 표준 편차 (Q에 사용)
measurement_noise_std = 5.0  # 측정 잡음(센서 잡음)의 표준 편차 (R에 사용)
num_steps = 300 # 총 시뮬레이션 스텝 수
accel_range = 0.05 # 가속도 최대 크기

# --- 시스템 상태 정의 ---
# 상태 벡터: [x_위치, y_위치, x_속도, y_속도]
A = np.array([[1, 0, dt, 0],
              [0, 1, 0, dt],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
B = np.array([[0.5 * dt**2, 0],
              [0, 0.5 * dt**2],
              [dt, 0],
              [0, dt]])
H = np.array([[1, 0, 0, 0], # 위치만 측정
              [0, 1, 0, 0]])
# Q: 프로세스 잡음 공분산. 모델이 얼마나 정확한지에 대한 불확실성.
Q = np.eye(4) * accel_noise_std**2
R = np.eye(2) * measurement_noise_std**2 # 측정 잡음 공분산

# --- 칼만 필터 초기 추정값 ---
x_est_initial = np.array([WIDTH / 2.0, HEIGHT / 2.0, 0.0, 0.0]) # 초기 추측값 (화면 중앙에서 시작)

# --- 초기 추정 공분산 ---
P_initial = np.eye(4) * 10.0 # 초기 불확실성 (크게 설정하여 초기 측정에 더 의존하도록)

# ==============================================================
# 단계 1: 제어 입력 시뮬레이션 (실제 경로 생성 없음)
# ==============================================================
print("단계 1: 제어 입력 시뮬레이션 중...")

control_inputs = []   # 사용된 제어 입력(임의 가속도)을 저장할 리스트

for k in range(num_steps):
    # --- 임의의 제어 입력 (가속도) 생성 ---
    ax = np.random.uniform(-accel_range, accel_range)
    ay = np.random.uniform(-accel_range, accel_range)
    u = np.array([ax, ay])
    control_inputs.append(u.copy())

print(f"단계 1 완료: {num_steps} 스텝의 제어 입력 생성됨.")

# ==============================================================
# 단계 2: 칼만 필터 적용 (이전 추정값으로부터 측정값 실시간 생성)
# ==============================================================
print("단계 2: 칼만 필터 적용 및 이전 추정 기반 측정값 생성 중...") # 설명 수정

estimated_states = []       # 각 스텝에서의 칼만 필터 상태 추정값을 저장할 리스트
predicted_states_for_viz = [] # 시각화를 위해 예측 상태(x_pred)를 저장할 리스트
measurements_for_viz = []   # 시각화를 위해 이 단계에서 생성된 측정값을 저장할 리스트

# 칼만 필터 상태 및 공분산 초기화
x_est = x_est_initial.copy() # x_est는 x_hat_{k-1|k-1}을 의미 (첫 스텝에서는 초기값)
P = P_initial.copy()         # P는 P_{k-1|k-1}을 의미

for k in range(num_steps):
    u_k = control_inputs[k]

    # --- 예측 단계 (Prediction Step) ---
    # x_est (x_hat_{k-1|k-1})를 사용하여 x_pred (x_hat_{k|k-1})를 계산
    x_pred = A @ x_est + B @ u_k
    P_pred = A @ P @ A.T + Q

    predicted_states_for_viz.append(x_pred.copy()) # 시각화를 위해 예측 상태 저장

    # --- 측정값 실시간 생성 (이전 추정값 기반) ---
    # 현재 x_est (즉, x_hat_{k-1|k-1})를 기반으로 측정 잡음을 추가하여 측정값을 생성합니다.
    measurement_noise = np.random.multivariate_normal([0, 0], R) # R의 크기에 맞게 2차원 잡음
    # H에 곱해지는 것이 x_est (이전 스텝의 칼만 필터로 보정된 값)
    measurement_k = H @ x_est + measurement_noise
    measurements_for_viz.append(measurement_k.copy()) # 시각화를 위해 저장

    # --- 갱신 단계 (Update Step) ---
    # 생성한 measurement_k와 예측값 x_pred를 사용합니다.
    y = measurement_k - H @ x_pred            # 측정 잔차 (Innovation) z_k - H @ x_hat_k|k-1
    S = H @ P_pred @ H.T + R                  # 측정 잔차 공분산
    K = P_pred @ H.T @ np.linalg.inv(S)       # 칼만 이득
    x_est_updated = x_pred + K @ y            # 상태 추정치 갱신 (이것이 x_hat_{k|k})
    P_updated = (np.eye(A.shape[0]) - K @ H) @ P_pred # 오차 공분산 갱신 (이것이 P_{k|k})

    # --- 결과 저장 및 다음 스텝 준비 ---
    estimated_states.append(x_est_updated.copy()) # 갱신된 추정값 (x_hat_{k|k}) 저장
    x_est = x_est_updated.copy() # 다음 반복을 위해 x_est를 x_hat_{k|k}로 업데이트
    P = P_updated.copy()         # 다음 반복을 위해 P를 P_{k|k}로 업데이트

print("단계 2 완료: 칼만 필터 적용 및 이전 추정 기반 측정값 생성 완료.") # 설명 수정

# ==============================================================
# 단계 3: 시각화
# ==============================================================
print("단계 3: 시각화 시작...")

running = True
frame_index = 0
simulation_paused = False # 시뮬레이션 일시정지 기능 추가

# 색상 정의
COLOR_BACKGROUND = (255, 255, 255)
COLOR_PREDICTED_PATH = (0, 0, 255) # 예측 경로 (파란색) - x_pred (x_hat_k|k-1)
COLOR_MEASUREMENT = (180, 180, 180) # 측정 위치 (회색)
COLOR_ESTIMATED = (255, 0, 0)     # 추정 위치 (빨간색) - x_est (x_hat_k|k)
COLOR_TEXT = (0, 0, 0)
COLOR_GRID = (220, 220, 220)

font_small = pygame.font.Font(None, 30)
font_large = pygame.font.Font(None, 50)

def draw_grid():
    for x in range(0, WIDTH, 50):
        pygame.draw.line(screen, COLOR_GRID, (x, 0), (x, HEIGHT))
    for y in range(0, HEIGHT, 50):
        pygame.draw.line(screen, COLOR_BACKGROUND, (0, y), (WIDTH, y)) # 오타 수정: COLOR_GRID

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE: # 스페이스바를 누르면 일시정지/재개
                simulation_paused = not simulation_paused
            if event.key == pygame.K_r and frame_index >= num_steps: # 'r'을 눌러 완료 후 재시작 (간단한 재시작)
                frame_index = 0
                # 상태 재초기화는 현재 시각화만 재시작 (데이터는 이전 시뮬레이션 유지)
                # 완전한 재시뮬레이션을 원한다면 단계 2 로직을 다시 실행해야 합니다.
                print("시각화 재시작 (데이터는 이전 시뮬레이션 유지)")


    screen.fill(COLOR_BACKGROUND)
    # draw_grid() # 격자 그리기 (선택 사항)

    if frame_index < num_steps:
        # 현재 프레임 데이터 가져오기
        predicted_state_k = predicted_states_for_viz[frame_index] # x_pred
        measurement_k = measurements_for_viz[frame_index]
        estimated_state_k = estimated_states[frame_index]     # x_est (updated)

        # 예측 위치 (파란색) - x_pred (x_hat_k|k-1)
        # 이 경로가 "측정의 기준"이 되는 것은 아니지만, 필터의 예측 단계를 보여줍니다.
        pred_x = int(np.clip(predicted_state_k[0], 0, WIDTH))
        pred_y = int(np.clip(predicted_state_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, COLOR_PREDICTED_PATH, (pred_x, pred_y), 7, 2) # 테두리만

        # 측정 위치 (회색)
        meas_x = int(np.clip(measurement_k[0], 0, WIDTH))
        meas_y = int(np.clip(measurement_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, COLOR_MEASUREMENT, (meas_x, meas_y), 6)

        # 추정 위치 (빨간색) - x_est (x_hat_k|k)
        est_x = int(np.clip(estimated_state_k[0], 0, WIDTH))
        est_y = int(np.clip(estimated_state_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, COLOR_ESTIMATED, (est_x, est_y), 8)

        # 궤적 선 그리기
        if frame_index > 0:
            prev_pred_x = int(np.clip(predicted_states_for_viz[frame_index-1][0], 0, WIDTH))
            prev_pred_y = int(np.clip(predicted_states_for_viz[frame_index-1][1], 0, HEIGHT))
            pygame.draw.line(screen, COLOR_PREDICTED_PATH, (prev_pred_x, prev_pred_y), (pred_x, pred_y), 1)

            prev_est_x = int(np.clip(estimated_states[frame_index-1][0], 0, WIDTH))
            prev_est_y = int(np.clip(estimated_states[frame_index-1][1], 0, HEIGHT))
            pygame.draw.line(screen, COLOR_ESTIMATED, (prev_est_x, prev_est_y), (est_x, est_y), 2)

        if not simulation_paused:
            frame_index += 1
    else: # frame_index >= num_steps
        # 완료 메시지
        text_surface = font_large.render("시뮬레이션 완료", True, (0, 100, 0))
        text_rect = text_surface.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text_surface, text_rect)
        restart_text = font_small.render("R 키로 시각화 재시작", True, COLOR_TEXT)
        screen.blit(restart_text, (text_rect.left, text_rect.bottom + 10))


    # 스텝 정보 및 일시정지 상태 표시
    status_text = f"스텝: {min(frame_index, num_steps)}/{num_steps}" # 완료 후에도 num_steps 초과하지 않도록
    if simulation_paused and frame_index < num_steps:
        status_text += " (일시정지 - SPACE 재개)"
    elif frame_index < num_steps:
        status_text += " (SPACE 일시정지)"

    text_surface = font_small.render(status_text, True, COLOR_TEXT)
    screen.blit(text_surface, (10, 10))

    pygame.display.flip()
    clock.tick(30) # 초당 30 프레임

pygame.quit()
print("시각화 종료 및 Pygame 종료됨.")

pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
단계 1: 제어 입력 시뮬레이션 중...
단계 1 완료: 300 스텝의 제어 입력 생성됨.
단계 2: 칼만 필터 적용 및 이전 추정 기반 측정값 생성 중...
단계 2 완료: 칼만 필터 적용 및 이전 추정 기반 측정값 생성 완료.
단계 3: 시각화 시작...
시각화 종료 및 Pygame 종료됨.


In [4]:
import pygame
import numpy as np

# --- Pygame 초기화 ---
pygame.init()
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D EKF 시뮬레이션 (일정 제어 입력 + 공기 저항)") # 창 제목 변경
clock = pygame.time.Clock()

# --- 시뮬레이션 파라미터 ---
dt = 1.0
accel_noise_std = 0.1 # 약간 줄여서 제어 입력의 효과를 더 잘 보도록 함
measurement_noise_std = 5.0
num_steps = 300
# accel_range = 0.05 # 무작위 범위 대신 고정값 사용
K_DRAG = 0.01  # 공기 저항 계수 (값을 조절하며 효과 확인)

# --- 일정한 제어 입력 (명령 가속도) 설정 ---
CMD_AX = 1.2  # x축 방향 일정한 명령 가속도
CMD_AY = 1  # y축 방향 일정한 명령 가속도 (0으로 두거나 다른 값으로 변경 가능)

# --- 시스템 상태 정의 (EKF에서는 f와 F로 대체됨) ---
H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0]])

Q_diag = np.array([
    (0.5 * dt**2)**2 * accel_noise_std**2,
    (0.5 * dt**2)**2 * accel_noise_std**2,
    (dt)**2 * accel_noise_std**2,
    (dt)**2 * accel_noise_std**2
])
Q = np.diag(Q_diag)
R = np.eye(2) * measurement_noise_std**2

# --- 초기 실제 상태 ---
initial_true_state = np.array([WIDTH / 8.0, HEIGHT / 8.0, 1.0, -0.5]) # 초기 위치와 속도 변경

# --- EKF 초기 추정값 ---
x_est_initial = np.array([WIDTH / 8.0 + 20, HEIGHT / 2.0 - 20, 0.0, 0.0]) # 초기 위치에 약간의 오차

# --- 초기 추정 공분산 ---
P_initial = np.eye(4) * 50.0 # 초기 불확실성을 약간 더 크게 설정

# --- EKF 관련 함수 정의 (이전과 동일) ---
def state_transition_function_f(x_prev, u_cmd, dt, k_d):
    px, py, vx, vy = x_prev
    ax_cmd, ay_cmd = u_cmd
    speed = np.sqrt(vx**2 + vy**2)
    if speed < 1e-6:
        ax_drag = 0
        ay_drag = 0
    else:
        ax_drag = -k_d * vx * speed
        ay_drag = -k_d * vy * speed
    ax_net = ax_cmd + ax_drag
    ay_net = ay_cmd + ay_drag
    px_next = px + vx * dt + 0.5 * ax_net * dt**2
    py_next = py + vy * dt + 0.5 * ay_net * dt**2
    vx_next = vx + ax_net * dt
    vy_next = vy + ay_net * dt
    return np.array([px_next, py_next, vx_next, vy_next])

def calculate_jacobian_F(x_eval, u_cmd, dt, k_d):
    px, py, vx, vy = x_eval
    F = np.eye(4)
    s = np.sqrt(vx**2 + vy**2)
    epsilon = 1e-6
    if s < epsilon:
        F[0, 2] = dt
        F[1, 3] = dt
        F[2, 2] = 1.0
        F[3, 3] = 1.0
    else:
        d_ax_drag_dvx = -k_d * (s + vx**2 / s)
        d_ax_drag_dvy = -k_d * (vx * vy / s)
        d_ay_drag_dvx = -k_d * (vx * vy / s)
        d_ay_drag_dvy = -k_d * (s + vy**2 / s)
        F[0, 2] = dt + 0.5 * d_ax_drag_dvx * dt**2
        F[0, 3] = 0.5 * d_ax_drag_dvy * dt**2
        F[1, 2] = 0.5 * d_ay_drag_dvx * dt**2
        F[1, 3] = dt + 0.5 * d_ay_drag_dvy * dt**2
        F[2, 2] = 1 + d_ax_drag_dvx * dt
        F[2, 3] = d_ax_drag_dvy * dt
        F[3, 2] = d_ay_drag_dvx * dt
        F[3, 3] = 1 + d_ay_drag_dvy * dt
    return F

# ==============================================================
# 단계 1: 실제 경로 및 제어 입력 시뮬레이션 (일정 제어 입력 사용)
# ==============================================================
print("단계 1: 실제 경로 및 제어 입력 시뮬레이션 중 (일정 제어 입력)...")
true_states = []
control_inputs = [] # 실제 사용된 제어 입력을 저장 (일정하지만 로깅 목적)
current_true_state = initial_true_state.copy()

for k in range(num_steps):
    # --- 일정한 제어 입력 (명령 가속도) 사용 ---
    u = np.array([CMD_AX, CMD_AY]) # 정의된 일정한 값 사용

    current_true_state_deterministic = state_transition_function_f(current_true_state, u, dt, K_DRAG)
    process_noise_vector = np.random.multivariate_normal(np.zeros(4), Q)
    current_true_state = current_true_state_deterministic + process_noise_vector

    true_states.append(current_true_state.copy())
    control_inputs.append(u.copy()) # 사용된 제어 입력 저장

print(f"단계 1 완료: {num_steps} 스텝의 실제 경로 및 제어 입력 생성됨 (일정 제어 입력).")

# ==============================================================
# 단계 2: EKF 적용 (측정값 실시간 생성) - 이전과 동일 로직
# ==============================================================
print("단계 2: EKF 적용 및 측정값 실시간 생성 중...")
estimated_states = []
measurements_for_viz = []

x_est = x_est_initial.copy()
P = P_initial.copy()

for k in range(num_steps):
    true_state_k = true_states[k]
    # u_k는 control_inputs 리스트에서 가져오거나, 매번 동일한 CMD_AX, CMD_AY를 사용해도 됨
    # 여기서는 단계 1에서 저장한 control_inputs[k]를 사용 (일관성)
    u_k = control_inputs[k]

    x_pred = state_transition_function_f(x_est, u_k, dt, K_DRAG)
    F_k = calculate_jacobian_F(x_est, u_k, dt, K_DRAG) # x_est (이전 추정치) 기준
    P_pred = F_k @ P @ F_k.T + Q

    measurement_noise = np.random.multivariate_normal(np.zeros(2), R)
    measurement_k = H @ true_state_k + measurement_noise
    measurements_for_viz.append(measurement_k.copy())

    y = measurement_k - H @ x_pred
    S = H @ P_pred @ H.T + R
    K = P_pred @ H.T @ np.linalg.inv(S)
    x_est = x_pred + K @ y
    P = (np.eye(4) - K @ H) @ P_pred

    estimated_states.append(x_est.copy())

print("단계 2 완료: EKF 적용 및 측정값 생성 완료.")


# ==============================================================
# 단계 3: 시각화 (제어 입력 값 표시 추가)
# ==============================================================
print("단계 3: 시각화 시작...")

running = True
frame_index = 0

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255, 255, 255))

    if frame_index < num_steps:
        true_state_k = true_states[frame_index]
        measurement_k = measurements_for_viz[frame_index]
        estimated_state_k = estimated_states[frame_index]

        true_x = int(np.clip(true_state_k[0], 0, WIDTH))
        true_y = int(np.clip(true_state_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, (0, 0, 255), (true_x, true_y), 8)

        meas_x = int(np.clip(measurement_k[0], 0, WIDTH))
        meas_y = int(np.clip(measurement_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, (180, 180, 180), (meas_x, meas_y), 6)

        est_x = int(np.clip(estimated_state_k[0], 0, WIDTH))
        est_y = int(np.clip(estimated_state_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, (255, 0, 0), (est_x, est_y), 8)
        
        if frame_index > 0:
            prev_true_x = int(np.clip(true_states[frame_index-1][0], 0, WIDTH))
            prev_true_y = int(np.clip(true_states[frame_index-1][1], 0, HEIGHT))
            pygame.draw.line(screen, (0, 0, 200), (prev_true_x, prev_true_y), (true_x, true_y), 1)
            
            prev_est_x = int(np.clip(estimated_states[frame_index-1][0], 0, WIDTH))
            prev_est_y = int(np.clip(estimated_states[frame_index-1][1], 0, HEIGHT))
            pygame.draw.line(screen, (200, 0, 0), (prev_est_x, prev_est_y), (est_x, est_y), 1)

        font = pygame.font.Font(None, 30)
        text = font.render(f"스텝: {frame_index+1}/{num_steps} (EKF, 일정 CMD)", True, (0, 0, 0))
        screen.blit(text, (10, 10))
        
        drag_text = font.render(f"K_DRAG: {K_DRAG:.3f}", True, (0,0,0))
        screen.blit(drag_text, (10, 40))
        
        # 일정한 명령 가속도 값 표시
        cmd_accel_text = font.render(f"CMD_AX: {CMD_AX:.2f}, CMD_AY: {CMD_AY:.2f}", True, (0,0,0))
        screen.blit(cmd_accel_text, (10, 70))


        frame_index += 1
    else:
        font = pygame.font.Font(None, 50)
        text = font.render("EKF 시뮬레이션 완료", True, (0, 100, 0))
        text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text, text_rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()
print("시각화 종료 및 Pygame 종료됨.")

단계 1: 실제 경로 및 제어 입력 시뮬레이션 중 (일정 제어 입력)...
단계 1 완료: 300 스텝의 실제 경로 및 제어 입력 생성됨 (일정 제어 입력).
단계 2: EKF 적용 및 측정값 실시간 생성 중...
단계 2 완료: EKF 적용 및 측정값 생성 완료.
단계 3: 시각화 시작...
시각화 종료 및 Pygame 종료됨.


In [1]:
import pygame
import numpy as np

# --- Pygame 초기화 ---
pygame.init()
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D EKF (거리 측정, 일정 CMD, 공기 저항)")
clock = pygame.time.Clock()

# --- 시뮬레이션 파라미터 ---
dt = 1.0
accel_noise_std = 0.1
measurement_noise_std = 10.0  # 거리 측정 잡음의 표준 편차 (단위: 픽셀 또는 해당 단위)
num_steps = 300
K_DRAG = 0.03
CMD_AX = 1.2
CMD_AY = 1

# --- 센서 위치 정의 (월드 좌표) ---
SENSOR_POS = np.array([WIDTH / 2.0, HEIGHT / 2.0]) # 화면 중앙에 센서 위치

# --- 시스템 상태 정의 ---
# Q: 프로세스 잡음 공분산
Q_diag = np.array([
    (0.5 * dt**2)**2 * accel_noise_std**2,
    (0.5 * dt**2)**2 * accel_noise_std**2,
    (dt)**2 * accel_noise_std**2,
    (dt)**2 * accel_noise_std**2
])
Q = np.diag(Q_diag)

# R: 측정 잡음 공분산 (거리는 스칼라이므로 1x1)
R = np.array([[measurement_noise_std**2]])

# --- 초기 실제 상태 ---
initial_true_state = np.array([WIDTH / 4.0, HEIGHT / 4.0, 0.8, 0.6])

# --- EKF 초기 추정값 ---
x_est_initial = np.array([WIDTH / 4.0 + 30, HEIGHT / 4.0 - 30, 0.0, 0.0])
P_initial = np.eye(4) * 100.0

# --- EKF 관련 함수 정의 ---
def state_transition_function_f(x_prev, u_cmd, dt, k_d):
    px, py, vx, vy = x_prev
    ax_cmd, ay_cmd = u_cmd
    speed = np.sqrt(vx**2 + vy**2)
    ax_drag, ay_drag = 0, 0
    if speed > 1e-6:
        ax_drag = -k_d * vx * speed
        ay_drag = -k_d * vy * speed
    ax_net = ax_cmd + ax_drag
    ay_net = ay_cmd + ay_drag
    px_next = px + vx * dt + 0.5 * ax_net * dt**2
    py_next = py + vy * dt + 0.5 * ay_net * dt**2
    vx_next = vx + ax_net * dt
    vy_next = vy + ay_net * dt
    return np.array([px_next, py_next, vx_next, vy_next])

def calculate_jacobian_F(x_eval, u_cmd, dt, k_d): # u_cmd는 현재 사용되지 않지만 인터페이스 일관성
    px, py, vx, vy = x_eval
    F = np.eye(4)
    s = np.sqrt(vx**2 + vy**2)
    epsilon = 1e-6
    if s < epsilon:
        F[0, 2] = dt; F[1, 3] = dt
        F[2, 2] = 1.0; F[3, 3] = 1.0
    else:
        d_ax_drag_dvx = -k_d * (s + vx**2 / s)
        d_ax_drag_dvy = -k_d * (vx * vy / s)
        d_ay_drag_dvx = d_ax_drag_dvy # 대칭
        d_ay_drag_dvy = -k_d * (s + vy**2 / s)
        F[0, 2] = dt + 0.5 * d_ax_drag_dvx * dt**2
        F[0, 3] = 0.5 * d_ax_drag_dvy * dt**2
        F[1, 2] = 0.5 * d_ay_drag_dvx * dt**2
        F[1, 3] = dt + 0.5 * d_ay_drag_dvy * dt**2
        F[2, 2] = 1 + d_ax_drag_dvx * dt
        F[2, 3] = d_ax_drag_dvy * dt
        F[3, 2] = d_ay_drag_dvx * dt
        F[3, 3] = 1 + d_ay_drag_dvy * dt
    return F

def h_measurement_function(x_state, sensor_pos_param):
    px, py = x_state[0], x_state[1]
    sx, sy = sensor_pos_param[0], sensor_pos_param[1]
    dist = np.sqrt((px - sx)**2 + (py - sy)**2)
    return dist # 스칼라 거리 값

def calculate_jacobian_Hj(x_eval, sensor_pos_param):
    px, py = x_eval[0], x_eval[1]
    sx, sy = sensor_pos_param[0], sensor_pos_param[1]
    dp_x = px - sx
    dp_y = py - sy
    dist_sq = dp_x**2 + dp_y**2
    epsilon = 1e-6
    if dist_sq < epsilon**2:
        return np.array([[0.0, 0.0, 0.0, 0.0]]) # 거리가 0에 가까우면 야코비안 0
    dist = np.sqrt(dist_sq)
    H_j = np.zeros((1, 4))
    H_j[0, 0] = dp_x / dist
    H_j[0, 1] = dp_y / dist
    return H_j

# ==============================================================
# 단계 1: 실제 경로 및 제어 입력 시뮬레이션
# ==============================================================
print("단계 1: 실제 경로 및 제어 입력 시뮬레이션 중...")
true_states = []
control_inputs = []
current_true_state = initial_true_state.copy()
for k in range(num_steps):
    u = np.array([CMD_AX, CMD_AY])
    current_true_state_deterministic = state_transition_function_f(current_true_state, u, dt, K_DRAG)
    process_noise_vector = np.random.multivariate_normal(np.zeros(4), Q)
    current_true_state = current_true_state_deterministic + process_noise_vector
    true_states.append(current_true_state.copy())
    control_inputs.append(u.copy())
print("단계 1 완료.")

# ==============================================================
# 단계 2: EKF 적용 (측정값 실시간 생성)
# ==============================================================
print("단계 2: EKF 적용 및 측정값 실시간 생성 중...")
estimated_states = []
measurements_for_viz = [] # 이제 스칼라 거리 값을 저장
x_est = x_est_initial.copy()
P = P_initial.copy()

for k in range(num_steps):
    true_state_k = true_states[k]
    u_k = control_inputs[k]

    # --- EKF 예측 단계 ---
    x_pred = state_transition_function_f(x_est, u_k, dt, K_DRAG)
    F_k = calculate_jacobian_F(x_est, u_k, dt, K_DRAG) # x_est는 이전 스텝의 x_est (x_{k-1|k-1})
    P_pred = F_k @ P @ F_k.T + Q

    # --- 측정값 실시간 생성 (거리 기반) ---
    true_dist = h_measurement_function(true_state_k, SENSOR_POS)
    measurement_noise_scalar = np.random.normal(0, measurement_noise_std)
    measurement_k = true_dist + measurement_noise_scalar # 스칼라 측정값
    measurements_for_viz.append(measurement_k)

    # --- EKF 갱신 단계 ---
    H_j_k = calculate_jacobian_Hj(x_pred, SENSOR_POS) # x_pred (x_{k|k-1})에서 H_j 계산
    z_pred = h_measurement_function(x_pred, SENSOR_POS) # x_pred로 예측 거리 계산
    
    y = measurement_k - z_pred  # 측정 잔차 (스칼라)
    
    # S = H_j_k @ P_pred @ H_j_k.T + R  # S는 1x1 행렬
    # S_inv = np.linalg.inv(S)
    # K = P_pred @ H_j_k.T @ S_inv # K는 4x1 벡터
    
    # S가 스칼라에 가까우므로, 좀 더 직접적인 계산
    S_val = (H_j_k @ P_pred @ H_j_k.T + R).item() # .item()으로 스칼라 값 추출
    
    if np.abs(S_val) < 1e-9: # S_val이 너무 작으면 (역수 취하기 어려움)
        K = np.zeros((4,1)) # 칼만 이득을 0으로 (측정값 사용 안 함)
    else:
        K = (P_pred @ H_j_k.T) / S_val # K는 4x1 벡터

    x_est = x_pred + K.flatten() * y # K를 1D로 만들고 스칼라 y와 곱셈 (브로드캐스팅)
                                     # 또는 K @ np.array([[y]])
    
    # 공분산 업데이트 (Joseph Form for numerical stability)
    I_KH = np.eye(4) - K @ H_j_k
    P = I_KH @ P_pred @ I_KH.T + K @ R @ K.T
    # P = (np.eye(4) - K @ H_j_k) @ P_pred # 간단한 형태

    estimated_states.append(x_est.copy())
print("단계 2 완료.")

# ==============================================================
# 단계 3: 시각화
# ==============================================================
print("단계 3: 시각화 시작...")
running = True
frame_index = 0

sensor_draw_x = int(SENSOR_POS[0])
sensor_draw_y = int(SENSOR_POS[1])

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255, 255, 255))

    if frame_index < num_steps:
        true_state_k = true_states[frame_index]
        measured_dist_k = measurements_for_viz[frame_index]
        estimated_state_k = estimated_states[frame_index]

        # 실제 위치 (파란색)
        true_x = int(np.clip(true_state_k[0], 0, WIDTH))
        true_y = int(np.clip(true_state_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, (0, 0, 255), (true_x, true_y), 8)

        # 추정 위치 (빨간색)
        est_x = int(np.clip(estimated_state_k[0], 0, WIDTH))
        est_y = int(np.clip(estimated_state_k[1], 0, HEIGHT))
        pygame.draw.circle(screen, (255, 0, 0), (est_x, est_y), 8)
        
        # 센서 위치 시각화 (녹색 사각형)
        pygame.draw.rect(screen, (0, 255, 0), (sensor_draw_x - 5, sensor_draw_y - 5, 10, 10))

        # 측정된 거리(스칼라)를 원으로 시각화 (회색)
        # 이 원은 "실제 객체는 이 원 어딘가에 있다"는 정보를 시각화 (노이즈 포함)
        pygame.draw.circle(screen, (180, 180, 180), (sensor_draw_x, sensor_draw_y), int(measured_dist_k), 1)

        if frame_index > 0:
            prev_true_x = int(np.clip(true_states[frame_index-1][0], 0, WIDTH))
            prev_true_y = int(np.clip(true_states[frame_index-1][1], 0, HEIGHT))
            pygame.draw.line(screen, (0, 0, 200), (prev_true_x, prev_true_y), (true_x, true_y), 1)
            
            prev_est_x = int(np.clip(estimated_states[frame_index-1][0], 0, WIDTH))
            prev_est_y = int(np.clip(estimated_states[frame_index-1][1], 0, HEIGHT))
            pygame.draw.line(screen, (200, 0, 0), (prev_est_x, prev_est_y), (est_x, est_y), 1)

        font = pygame.font.Font(None, 30)
        text = font.render(f"스텝: {frame_index+1}/{num_steps} (EKF, 거리측정)", True, (0, 0, 0))
        screen.blit(text, (10, 10))
        drag_text = font.render(f"K_DRAG: {K_DRAG:.3f}", True, (0,0,0))
        screen.blit(drag_text, (10, 40))
        cmd_accel_text = font.render(f"CMD_AX: {CMD_AX:.2f}, CMD_AY: {CMD_AY:.2f}", True, (0,0,0))
        screen.blit(cmd_accel_text, (10, 70))
        frame_index += 1
    else:
        font = pygame.font.Font(None, 50)
        text = font.render("EKF 시뮬레이션 완료", True, (0, 100, 0))
        text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text, text_rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()
print("시각화 종료 및 Pygame 종료됨.")

pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
단계 1: 실제 경로 및 제어 입력 시뮬레이션 중...
단계 1 완료.
단계 2: EKF 적용 및 측정값 실시간 생성 중...
단계 2 완료.
단계 3: 시각화 시작...
시각화 종료 및 Pygame 종료됨.
